In [1]:
import pandas as pd
import requests
import json
import re

Your task is to add 2 columns to the dataset I provided, one column for population count, and another for the Deprivation Index.

In [2]:
def getPop(lat,lng):
    '''
    returns population if obtained, or a negative integer that has absolute value being the negative response code
    '''
    url = "https://koordinates.com/services/query/v1/vector.json"
    params = {
        'key': 'd34f32af5b7e41ccabc7aa9c67a76a43',
        'layer': 104612,
        'x': lng,
        'y': lat    
    }
    response = requests.get(url,params=params)
    if response.status_code != 200:
        return pd.Series({'Population18': -response.status_code})
    properties = response.json()['vectorQuery']['layers']['104612']['features'][0]['properties']
    return pd.Series({'Population18': properties['C18_CURPop']})


In [3]:
# dfDI: dataframe for deprive index
dfDI = pd.read_excel('otago730395.xlsx')
dfDI = dfDI.drop(['NZDep2018_Score', 'SA22018_code', 'SA22018_name'], axis=1)

In [4]:
dfDI.head()

,SA12018_code,NZDep2018,URPopnSA1_2018
0,7000000,10.0,141
1,7000001,10.0,114
2,7000002,NaN,0
3,7000003,10.0,225
4,7000004,9.0,138


In [5]:
df = pd.read_csv('Dataset.csv')

In [6]:
df = df.merge(dfDI, left_on = 'SA1', right_on = 'SA12018_code', how='left')

In [7]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,SA12018_code,NZDep2018,URPopnSA1_2018
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,7009770,6.0,174
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,7009991,1.0,129
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,7009991,1.0,129
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,7007871,2.0,120
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,7008902,9.0,231


In [8]:
df.shape

(1051, 18)

In [9]:

df = df.drop(['SA12018_code'], axis=1)

In [10]:
df['Population18'] = df.apply(lambda r: getPop(r['Latitude'], r['Longitude']), axis=1)

In [11]:
# check presence of negative response code
df.loc[df['Population18'] < 0]

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18


In [12]:
# check whether population from 2 resources are coherent
if (df.loc[df['Population18'] != df['URPopnSA1_2018']]):
    df = df.drop(['URPopnSA1_2018'], axis=1)
    print("Succeed")
else:
    df.loc[df['Population18'] != df['URPopnSA1_2018']]

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [13]:
df.tail()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18
1046,4,1.0,"19 Landscape Road, Auckland",1368 m²,670000,-36.899255,174.761165,7005464,54,18,15,24,21,27,Mount Eden,1.0,159,159
1047,6,1.0,"56 Galway Street, Auckland",607 m²,1200000,-36.844933,174.770001,7005497,15,27,24,15,18,30,Auckland Central,6.0,129,129
1048,5,3.0,"28A Hayr Road, Auckland",453 m²,1250000,-36.912242,174.756726,7007758,36,30,45,21,24,21,Three Kings,6.0,180,180
1049,5,2.0,"27 Market Road, Auckland",1854 m²,5300000,-36.879665,174.787668,7005745,48,18,12,15,36,45,Remuera,1.0,174,174
1050,3,1.0,"23 William Avenue, Auckland",806 m²,1665000,-36.897104,174.800171,7005917,54,33,27,27,15,30,Greenlane,4.0,192,192


###Start cleaning up data

In [14]:
df.loc[df.isnull().any(axis=1)]

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18
309,4,NaN,"14 Hea Road Hobsonville, Auckland",214,1250000,-36.798371,174.647430,7002267,60,66,60,24,24,18,Hobsonville,2.0,252,252
311,4,NaN,"16 Hea Road Hobsonville, Auckland",245,1100000,-36.798371,174.647430,7002267,60,66,60,24,24,18,Hobsonville,2.0,252,252
568,1,1.0,"14 Te Rangitawhiri Road Great Barrier Island, ...",2141,740000,-36.197282,175.416921,7001131,27,6,6,18,39,60,NaN,9.0,156,156


In [15]:
# replace the only NaN suburb with same suburb name as another house in the same street
df.loc[df['Suburbs'].isnull(), 'Suburbs'] = 'Great Barrier Island (Aotea Island)'

In [16]:
df.loc[df['Suburbs']=='Great Barrier Island (Aotea Island)']

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18
496,3,1.0,"236 Kaiaraara Bay Road Great Barrier Island, A...",5638,580000,-36.177655,175.359070,7001130,39,9,18,24,24,42,Great Barrier Island (Aotea Island),9.0,153,153
568,1,1.0,"14 Te Rangitawhiri Road Great Barrier Island, ...",2141,740000,-36.197282,175.416921,7001131,27,6,6,18,39,60,Great Barrier Island (Aotea Island),9.0,156,156
569,5,3.0,"349 Blind Bay Road Great Barrier Island, Auckland",3953,920000,-36.257895,175.436448,7001131,27,6,6,18,39,60,Great Barrier Island (Aotea Island),9.0,156,156
731,2,2.0,"8 Omanawa Lane Great Barrier Island, Auckland",8638,650000,-36.305955,175.492424,7001135,30,21,21,21,39,69,Great Barrier Island (Aotea Island),9.0,201,201


In [17]:
df.loc[df['Suburbs'] == 'Hobsonville']

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18
223,4,2.0,"10 Eyton Kay Road Hobsonville, Auckland",161,860000,-36.795951,174.655930,7002301,51,36,57,18,6,12,Hobsonville,1.0,174,174
309,4,NaN,"14 Hea Road Hobsonville, Auckland",214,1250000,-36.798371,174.647430,7002267,60,66,60,24,24,18,Hobsonville,2.0,252,252
310,4,2.0,"12 Hea Road Hobsonville, Auckland",191,530000,-36.798371,174.647430,7002267,60,66,60,24,24,18,Hobsonville,2.0,252,252
311,4,NaN,"16 Hea Road Hobsonville, Auckland",245,1100000,-36.798371,174.647430,7002267,60,66,60,24,24,18,Hobsonville,2.0,252,252
449,5,4.0,"10 Mantis Lane Hobsonville, Auckland",336,545000,-36.801329,174.666149,7002304,33,30,24,15,12,18,Hobsonville,4.0,135,135
460,4,2.0,"12 Williams Road Hobsonville, Auckland",450,1125000,-36.800550,174.645182,7002271,12,12,9,6,9,75,Hobsonville,7.0,129,129
666,3,2.0,"17 Kanuka Road Hobsonville, Auckland",259,920000,-36.793782,174.660944,7002295,45,24,48,15,6,12,Hobsonville,2.0,147,147


In [18]:
df.loc[df['Bedrooms'] ==4].describe()

,Bedrooms,Bathrooms,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,NZDep2018,URPopnSA1_2018,Population18
count,353.0,351.000000,3.530000e+02,353.000000,353.000000,3.530000e+02,353.000000,353.000000,353.000000,353.000000,353.000000,353.000000,353.000000,353.000000,353.000000
mean,4.0,2.213675,1.455133e+06,-36.903300,174.793559,7.006392e+06,49.181303,27.348442,26.974504,24.960340,23.830028,30.645892,4.368272,183.127479,183.127479
std,0.0,0.690702,1.021287e+06,0.127020,0.113575,2.628052e+03,26.052634,17.475481,19.024115,11.303087,10.002244,28.538388,2.801240,75.654817,75.654817
min,4.0,1.000000,3.000000e+05,-37.262183,174.456874,7.001308e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,18.000000,18.000000
25%,4.0,2.000000,8.900000e+05,-36.953018,174.711737,7.004427e+06,33.000000,15.000000,15.000000,18.000000,18.000000,21.000000,2.000000,141.000000,141.000000
50%,4.0,2.000000,1.175000e+06,-36.895269,174.796671,7.006418e+06,45.000000,24.000000,24.000000,24.000000,24.000000,27.000000,4.000000,171.000000,171.000000
75%,4.0,3.000000,1.700000e+06,-36.857708,174.881395,7.008270e+06,57.000000,33.000000,36.000000,30.000000,30.000000,39.000000,6.000000,213.000000,213.000000
max,4.0,5.000000,9.000000e+06,-36.409687,175.083269,7.011028e+06,201.000000,138.000000,168.000000,114.000000,81.000000,483.000000,10.000000,789.000000,789.000000


In [19]:
# replace the number of bathroom for NaN values to be 2 since it's the only value in the suburb with 4 bedrooms, also the global median and mean when there are 4 bedrooms is exactly or close to 2.
df.loc[df['Bathrooms'].isnull(), 'Bathrooms'] = 2

In [20]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,6.0,174,174
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,1.0,129,129
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,1.0,129,129
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,2.0,120,120
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,9.0,231,231


In [21]:
df.dtypes

Bedrooms            int64
Bathrooms         float64
Address            object
Land area          object
CV                  int64
Latitude          float64
Longitude         float64
SA1                 int64
0-19 years          int64
20-29 years         int64
30-39 years         int64
40-49 years         int64
50-59 years         int64
60+ years           int64
Suburbs            object
NZDep2018         float64
URPopnSA1_2018      int64
Population18        int64
dtype: object

In [22]:
df['Land area'].unique

<bound method Series.unique of 0           714
1           564
2           626
3            65
4           601
         ...   
1046    1368 m²
1047     607 m²
1048     453 m²
1049    1854 m²
1050     806 m²
Name: Land area, Length: 1051, dtype: object>

In [23]:
df['Land area'] = df['Land area'].apply(lambda x: int(re.match("(\d+)( .*)?", x).group(1)))

In [24]:
df['Bathrooms'] = df['Bathrooms'].astype('int64')

In [25]:
df.describe()

,Bedrooms,Bathrooms,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,NZDep2018,URPopnSA1_2018,Population18
count,1051.000000,1051.000000,1051.000000,1.051000e+03,1051.000000,1051.000000,1.051000e+03,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000
mean,3.777355,2.073264,856.989534,1.387521e+06,-36.893715,174.799325,7.006319e+06,47.549001,28.963844,27.042816,24.125595,22.615604,29.360609,5.063749,179.914367,179.914367
std,1.169412,0.992044,1588.156219,1.182939e+06,0.130100,0.119538,2.591262e+03,24.692205,21.037441,17.975408,10.942770,10.210578,21.805031,2.913471,71.059280,71.059280
min,1.000000,1.000000,40.000000,2.700000e+05,-37.265021,174.317078,7.001130e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,3.000000
25%,3.000000,1.000000,321.000000,7.800000e+05,-36.950565,174.720779,7.004416e+06,33.000000,15.000000,15.000000,18.000000,15.000000,18.000000,2.000000,138.000000,138.000000
50%,4.000000,2.000000,571.000000,1.080000e+06,-36.893132,174.798575,7.006325e+06,45.000000,24.000000,24.000000,24.000000,21.000000,27.000000,5.000000,174.000000,174.000000
75%,4.000000,3.000000,825.000000,1.600000e+06,-36.855789,174.880944,7.008384e+06,57.000000,36.000000,33.000000,30.000000,27.000000,36.000000,8.000000,210.000000,210.000000
max,17.000000,8.000000,22240.000000,1.800000e+07,-36.177655,175.492424,7.011028e+06,201.000000,270.000000,177.000000,114.000000,90.000000,483.000000,10.000000,789.000000,789.000000


In [26]:
df.loc[df['Land area']>15000]
# there doesn't seem to be any unusual data for land area

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18
228,4,2,"74 Tarrant Road Waimauku, Auckland",15230,1325000,-36.733890,174.456874,7001370,36,12,9,24,21,18,Helensville,2.0,123,123
519,5,3,"93 Popes Road Takanini, Auckland",20226,2975000,-37.028973,174.931330,7010010,51,24,21,18,27,30,Takanini,9.0,174,174
566,3,2,"10 Goudie Road Helensville, Auckland",22240,1075000,-36.688850,174.388196,7001262,45,12,15,27,27,39,Helensville,3.0,165,165
662,3,1,"358 Mangawhai Road Wellsford, Auckland",15550,550000,-36.228742,174.545810,7001139,48,30,21,21,33,45,Wellsford,5.0,195,195


In [27]:
df.loc[df['CV']>6000000] 

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018,URPopnSA1_2018,Population18
183,4,4,"9 Darwin Lane Remuera, Auckland",2718,9000000,-36.871836,174.813689,7005974,27,12,6,18,18,36,Remuera,1.0,114,114
184,4,4,"9 Darwin Lane Remuera, Auckland",2718,9000000,-36.871836,174.813689,7005974,27,12,6,18,18,36,Remuera,1.0,114,114
266,6,3,"34d Arney Crescent Remuera, Auckland",602,6400000,-36.869937,174.788921,7005846,36,6,6,18,21,30,Remuera,1.0,129,129
306,2,2,"223 Green Lane West Epsom, Auckland",123,18000000,-36.892772,174.779791,7005755,3,9,15,6,15,21,Epsom,4.0,60,60
421,5,4,"15 Lammermoore Drive St Heliers, Auckland",1998,7000000,-36.863985,174.851967,7006417,51,27,9,21,42,51,St Heliers,1.0,192,192
441,4,3,"7-9 Glen Atkinson Street St Heliers, Auckland",1750,6500000,-36.866175,174.849780,7006420,42,21,12,21,39,39,St Heliers,1.0,174,174
572,5,6,"259 St Heliers Bay Road St Heliers, Auckland",2268,6600000,-36.863925,174.853299,7006423,39,18,15,21,39,30,St Heliers,1.0,165,165
582,4,5,"379a Gordons Road Waiheke Island, Auckland",641,6400000,-36.829057,175.074283,7003389,30,9,18,27,39,42,Waiheke Island,4.0,168,168
591,6,4,"410 Pukapuka Road Mahurangi, Auckland",607,6200000,-36.474573,174.693415,7001299,27,15,15,21,27,33,Pohuehue,5.0,135,135
1029,6,4,"43 Arney Road, Auckland",3062,12250000,-36.872514,174.788572,7005696,60,12,15,27,24,33,Remuera,1.0,168,168


In [28]:
df.loc[df['CV'] == 18000000, 'CV'] = 1800000
# the price looks suspicious with respect to the land area and other parameters

In [29]:
df.drop_duplicates(inplace=True)

In [30]:
df.to_csv('prepared data.csv')